In [1]:
%display latex
import sympy
Parallelism().set(nproc=8)

We are using SageMath version 9.2 or later

In [2]:
version()

'SageMath version 9.2, Release Date: 2020-10-24'

In [3]:
load("non_unimod_dim_3.sage")

# The case of $G_c$, $c < 0$

We write
$$
c = 1 - c_1^2, \qquad \text{where } c_1 > 1
$$

In [4]:
c1 = var("c1")
assume(c1>1)
c = 1 - c1^2
G = non_unimod_dim_3('G_c', c)
L = G.frame_left_inv()
L0, L1, L2 = L
R = G.frame_right_inv()
R0, R1, R2 = R

Checking that the structure coefficients coincide with the ones of $\mathfrak g_c$

In [5]:
L0.bracket(L1).display(L)

[L_0,L_1] = 0

In [6]:
L2.bracket(L0).display(L)

[L_2,L_0] = L_1

In [7]:
L2.bracket(L1).display(L)

[L_2,L_1] = (c1^2 - 1) L_0 + 2 L_1

## Left invariant metrics: $g = g_{\mu, \nu}$

We introduce the left invariant metric `g`, which in the frame `L` is given by the matrix
$$
g_{\mu,\nu} = 
\begin{pmatrix}
1 & 0 & 0 \\
0 & \mu & 0 \\
0 & 0 & \nu
\end{pmatrix} \qquad 0 < \mu \le |c| \text{ and } 0 < \nu
$$

In [8]:
mu, nu = var('mu nu')
assume(0<mu,mu<=-c)
assume(0<nu)
g = G.left_inv_metric([1,0,0,0,mu,0,0,0,nu],name=r'g_{\mu,\nu}')
g.comp(L)[:]

[ 1  0  0]
[ 0 mu  0]
[ 0  0 nu]

We verify that the right invariant vector fields are Killing fields.

In [9]:
for X in R:
    display(g.lie_derivative(X).display(L))

0

0

0

We compute the intersecion of the orthogonal Lie algebra $\mathfrak{so}(g_{\mu, \nu}) \simeq \mathfrak{so}(3)$ with the pseudo-orthogonal Lie algebra with respecto to the Ricci tensor, $\mathfrak{so}(\operatorname{Ric}) \simeq \mathfrak{so}(2, 1)$

In [10]:
g_mat = mat_SR(g.comp(L)[:])
Ric = g.ricci()
Ric_mat = mat_SR(Ric.comp(L)[:]).simplify_full()
g_mat, Ric_mat

(
[ 1  0  0]  [              1/2*(c1^4 - 2*c1^2 - mu^2 + 1)/(mu*nu)                                             -2*mu/nu                                                    0]
[ 0 mu  0]  [                                            -2*mu/nu            -1/2*(c1^4 - 2*c1^2 - mu^2 + 8*mu + 1)/nu                                                    0]
[ 0  0 nu], [                                                   0                                                    0 -1/2*(c1^4 - 2*c1^2 + 2*(c1^2 + 3)*mu + mu^2 + 1)/mu]
)

In [11]:
A_list = []
for i in range(3):
    for j in range(3):
        A_list += [var('a'+str(i)+str(j))]
A = matrix(3,3,A_list); A

[a00 a01 a02]
[a10 a11 a12]
[a20 a21 a22]

Equations for $A \in \mathfrak{so}(g_{\mu, \nu})$:

In [12]:
sols_so3 = solve((A.transpose() * g_mat + g_mat * A).list(), A.list(), algorithm='sympy')
A_so3 = A.subs(sols_so3); A_so3

[         0    -a10*mu    -a20*nu]
[       a10          0 -a21*nu/mu]
[       a20        a21          0]

Equations for `A_{so3}` be also an element of $\mathfrak{so}(\operatorname{Ric})$

In [13]:
sols_so3_Ric = solve((A_so3.transpose() * Ric_mat + Ric_mat * A_so3).list(), [a10,a20,a21], algorithm='sympy')
A_so3_Ric = A_so3.subs(sols_so3_Ric); A_so3_Ric

[0 0 0]
[0 0 0]
[0 0 0]

Therefore, the Lie algebra of the isotropy group of the full isometry group is trivial.

## Index of symmetry

Since the isometry group has dimension $3$, if the index of symmery is non-trivial, we will find a left invariant vector field $X$ such that $X_e \neq 0$ and $(\nabla X)_e = 0$

In [14]:
a0, a1, a2 = var('a0 a1 a2')
R_gen = G.right_inv_vect_field(a0,a1,a2,name=r'R_{gen}')
R_gen.display(R)

R_{gen} = a0 R_0 + a1 R_1 + a2 R_2

Define the Levi-Civita connection

In [15]:
nabla = g.connection()

And take the derivative of a generic right invariant vector field.

In [16]:
nablaR_gen = nabla(R_gen).at(G.zero)
ind_sym_eqs = nablaR_gen.comp()[:].factor()
ind_sym_eqs

[                                       0                   1/2*(c1^2 - mu - 1)*a2                  -1/2*(c1^2 + mu - 1)*a1]
[              -1/2*(c1^2 - mu - 1)*a2/mu                                        0 -1/2*(a0*c1^2 + a0*mu + 4*a1*mu - a0)/mu]
[               1/2*(c1^2 + mu - 1)*a1/nu  1/2*(a0*c1^2 + a0*mu + 4*a1*mu - a0)/nu                                        0]

### Case $\mu \neq |c| = c_1^2 - 1$

In this case, clearly $a_1 = a_2$ and hence $a_0 = 0$.

In [17]:
ind_sym_eqs.subs(a1=0,a2=0).factor()

[                         0                          0                          0]
[                         0                          0 -1/2*(c1^2 + mu - 1)*a0/mu]
[                         0  1/2*(c1^2 + mu - 1)*a0/nu                          0]

Therefore, the index of symmetry is trivial when $\mu \neq |c|$.

### Case $\mu = |c| = c_1^2 - 1$

In [18]:
ind_sym_eqs1 = ind_sym_eqs.subs(mu=c1^2-1).factor()
ind_sym_eqs1

[                               0                                0            -a1*(c1 + 1)*(c1 - 1)]
[                               0                                0                       -a0 - 2*a1]
[         a1*(c1 + 1)*(c1 - 1)/nu (a0 + 2*a1)*(c1 + 1)*(c1 - 1)/nu                                0]

In [19]:
solve(ind_sym_eqs1.list(), [a0,a1,a2], algorithm='sympy')

[{a0: 0, a1: 0}]

Here, we can see that $a_0 = a_1 = 0$ and thus, the distribution of symmetry is spanned by the left invariant fielf $L_2$.